In [ ]:
import pandas as pd
import numpy as np
import sqlite3
pd.set_option('display.max_rows', 2000)

: 

# TUL

In [ ]:
# ETL TUL Utilisation data
tul_util_df = pd.read_excel("data/TULNetRateByTrain.xlsx")
tul_util_df["COMMENCE_DUMPING"] = pd.to_datetime(tul_util_df["COMMENCE_DUMPING"])
tul_util_df["COMPLETE_DUMPING"] = pd.to_datetime(tul_util_df["COMPLETE_DUMPING"])
tul_util_df = tul_util_df[["TUL", "COMMENCE_DUMPING", "COMPLETE_DUMPING"]].rename(columns={'COMMENCE_DUMPING': 'start', 'COMPLETE_DUMPING': 'end'})

In [ ]:
# ETL TUL APLUS data
tul_delay_df = pd.read_csv("tul_aplus_delays_fully_categorised.csv")
tul_delay_df["StartTime"] = pd.to_datetime(tul_delay_df["StartTime"])
tul_delay_df["EndTime"] = pd.to_datetime(tul_delay_df["EndTime"])
tul_delay_df["TUL"] = np.where(
    # assign TULs to each delay
    (
        (tul_delay_df["EquipmentName"].str.startswith("TUL 1")) |\
        (tul_delay_df["EquipmentName"].str.startswith("TUL601")) | \
         tul_delay_df["EquipmentName"].str.startswith("InCircuit1")
    ), 
    "TUL1",
    np.where(
        (
            (tul_delay_df["EquipmentName"].str.startswith("TUL 2")) | \
            (tul_delay_df["EquipmentName"].str.startswith("TUL602")) | \
             tul_delay_df["EquipmentName"].str.startswith("InCircuit2")
        ),
        "TUL2",
        "TUL3"
    )
)
tul_delay_df = tul_delay_df[["TUL", "StartTime", "EndTime", "DelayCategory"]].rename(columns={'StartTime': 'start', 'EndTime': 'end'})


In [ ]:

# Create time buckets crossed with TULs
RANGE_START = "2023-10-01"
RANGE_END = "2023-11-01"
TIME_WINDOW_MINS = 5
time_buckets = pd.date_range(RANGE_START, RANGE_END, freq=f'{TIME_WINDOW_MINS} min')
time_bucket_df = pd.DataFrame({'start': time_buckets})
time_bucket_df["end"] = time_bucket_df["start"] + pd.Timedelta(TIME_WINDOW_MINS, 'minute')
time_bucket_tul_df = pd.merge(time_bucket_df, pd.DataFrame(['TUL1', 'TUL2', 'TUL3'], columns=['TUL']), how='cross')

# Filter dfs to required range
tul_util_df = tul_util_df[
    (tul_util_df["start"] >= pd.to_datetime(RANGE_START)) &
    (tul_util_df["end"] <= pd.to_datetime(RANGE_END))
]
tul_delay_df = tul_delay_df[
    (tul_delay_df["start"] >= pd.to_datetime(RANGE_START)) &
    (tul_delay_df["end"] <= pd.to_datetime(RANGE_END))
]

# Perform range join on time buckets and util/delays
# NOTE: range joining on time is a pain in pandas, so using SQL to do it
conn = sqlite3.connect(':memory:')
time_bucket_tul_df.to_sql('time_buckets_per_tul', conn, index=False)
tul_util_df.to_sql('tul_transactions', conn, index=False)
tul_delay_df.to_sql('tul_delays', conn, index=False)
query = '''
    select
        A.start as time_bucket,
        A.TUL as tul,
        (julianday(min(A.end, B.end))-julianday(max(A.start, B.start)))*1440.0 as usage_time,
        (julianday(min(A.end, C.end))-julianday(max(A.start, C.start)))*1440.0 as delay_time,
        C.DelayCategory as delay_category
    from
        time_buckets_per_tul A
        left join
            tul_transactions B on (A.TUL = B.TUL) & ((julianday(min(A.end, B.end))-julianday(max(A.start, B.start)))*1440.0 > 0)
        left join
            tul_delays C on (A.TUL = C.TUL) & ((julianday(min(A.end, C.end))-julianday(max(A.start, C.start)))*1440.0 > 0)
'''
merged_df = pd.read_sql_query(query,conn)

# Determine status of TUL
merged_df["status"] = np.where(
    merged_df["usage_time"] >= TIME_WINDOW_MINS/2, # must be used greater than half the interval (same definition as used last year)
    "used",
    np.where(
        ((merged_df["delay_category"] == "Maintenance/inspection") & (merged_df["delay_time"] >= 2.5)) | ((merged_df["delay_category"] == "Standby_due_to_failure") & (merged_df["delay_time"] >= 2.5)),
        "unavailable",
        "idle"
    )
)

# Stock Pile Transaction

In [ ]:
run_query(  '''
select DATE(transaction_datetime) as transaction_date, stockpile_name, balance_wmt, transaction_datetime
from SBX_AA_OPERATIONS_ARTIFICIALINTELLIGENCE.STG_ARTIFICIALINTELLIGENCE.kmc_stg_delta__inventory_stockpile_transactions
where stockpile_type = 'PortStockpile' and transaction_datetime > '2024-01-01' and stockpile_name NOT LIKE '__ -%'
qualify row_number() over (partition by stockpile_name, DATE(transaction_datetime) order by transaction_datetime desc) = 1
order by stockpile_name,  DATE(transaction_datetime)
'''
)

In [ ]:
df = pd.read_csv("StockpileTransactions.csv")
# bucketed time
times = pd.date_range(RANGE_START, RANGE_END, freq=f'1d')
stockpiles = [f"{canyon}{number+1}" for canyon in ["B","C","D","E","F"] for number in range(6)]
df = df[df["STOCKPILENAME"].isin(stockpiles)]
df["TRANSACTIONDATETIME"] = pd.to_datetime(df["TRANSACTIONDATETIME"], dayfirst=True)
df = df[(df["TRANSACTIONDATETIME"] >= pd.to_datetime(RANGE_START)) & (df["TRANSACTIONDATETIME"] < pd.to_datetime(RANGE_END))]
df["date"] = df["TRANSACTIONDATETIME"].dt.date
df = df.sort_values("date")
df = df.groupby(["STOCKPILENAME", "date"]).agg(['last']).stack().reset_index()[["STOCKPILENAME", "date", "BALANCEWMT"]]
df = df.groupby("date")["BALANCEWMT"].sum().reset_index()
vals = df["BALANCEWMT"].quantile([0.33, 0.66])
df["Stockpile_health"] = np.where(
    df["BALANCEWMT"]>=vals[0.66],
    "High",
    np.where(
        df["BALANCEWMT"]>=vals[0.33],
        "Medium",
        "Low"
    )
)
df